In [4]:
# !pip3 install google.generativeai
# !pip3 install clingo
# !pip3 install vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 31.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.7/333.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 66.2 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# import ssl
# print(ssl.OPENSSL_VERSION)

In [22]:
import vertexai
from vertexai.preview.language_models import ChatModel, InputOutputTextPair, TextGenerationModel
from google.oauth2 import service_account
# connect to vertex ai, to be replaced with the respective account credentials
mycredentials = service_account.Credentials.from_service_account_file('aerial-grid-415517-361a194b7eef.json')
vertexai.init(project='aerial-grid-415517', location='us-central1', credentials=mycredentials)
import clingo
import re

In [23]:
def add_argument(file, argument):
    with open(file, 'r') as f:
        prompt = f.read()
        prompt_with_argument = prompt.replace("!!!!!!!!!!!!!!!!!!!!", argument)
        return prompt_with_argument

In [24]:
def copy_file_content(original_file, new_file):
    with open(original_file, 'r') as src:
        content = src.read()
    with open(new_file, 'w') as dst:
        dst.write(content)

In [25]:
def filter(output_file):
    with open(output_file, 'r') as output:
        lines = [line.strip() for line in output.readlines() if line.strip().startswith(('pos(', 'neg(', 'relation(', 'compare(')) or 'is_good_because' in line]
        with open(output_file, 'w') as output:
            output.write('\n'.join(lines))

In [26]:
def convert_number_to_words(s):
    num_words = {
        '0': 'zero', '1': 'one', '2': 'two', '3': 'three', '4': 'four',
        '5': 'five', '6': 'six', '7': 'seven', '8': 'eight', '9': 'nine'
    }
    # Replace each digit individually
    return ''.join(num_words.get(char, char) for char in s)

def replace_spaces_with_underscores_in_identifiers(s):
    # Precompile regular expression patterns for performance
    pattern_identifier = re.compile(r'\b([a-z_][a-z0-9_]*)\s+([a-z0-9_]+)\b', re.IGNORECASE)
    
    # Replace spaces between words that are likely part of an identifier with underscores
    def repl(m):
        return m.group(1) + '_' + m.group(2)

    # Apply the replacement repeatedly until no further replacements are made
    while True:
        s_new = pattern_identifier.sub(repl, s)
        if s_new == s:
            break
        s = s_new

    return s

def remove_unnecessary_characters(s):
    # First, remove specified unnecessary characters, keeping the dash to handle it separately
    s = re.sub(r'[\'`*<>~%$‘’"/]', '', s)
    # Replace "-" with "_", but avoid replacing it within " :- "
    # This uses a negative lookbehind and lookahead to ensure we don't replace dashes in " :- "
    s = re.sub(r'(?<!:)-', '_', s)  # Replace "-" with "_" if "-" is not immediately preceded by ":"
    s = re.sub(r'-(?! )', '_', s)  # Replace "-" with "_" if "-" is not immediately followed by a space, handling the case after ":-"
    
    return s

def remove_all_but_final_period(line):
    # Temporarily replace ":-" with a placeholder
    placeholder = "TEMP_COLON_HYPHEN"
    line = line.replace(':-', placeholder)

    # Remove all periods except the final one if it exists
    parts = line.rsplit('.', 1)
    if len(parts) == 2:
        line = parts[0].replace('.', '') + '.' + parts[1]
    else:
        line = parts[0].replace('.', '')

    # Restore ":-"
    line = line.replace(placeholder, ':-')
    return line

def filter_asp_code(asp_code):
    filtered_lines = []
    seen_lines = set()
    allowed_prefixes_pattern = re.compile(r'\b(neg|pos|relation)\(')

    for line in asp_code.split('\n'):
        # Add the processed line to the set of seen lines
        seen_lines.add(line)
        # Remove Non ASP code parts and special character
        if re.match(r'^\*\*|```asp', line):
            continue
        # Remove lines not in ASP format
        if not re.match(r'^[a-zA-Z0-9_].*', line):
            continue
        # Replace spaces with underscores in identifiers
        line = replace_spaces_with_underscores_in_identifiers(line)
        # remove lines with the word section in it
        if "section" in line.lower():
            continue
        if not "." in line.lower():
            continue
        if not allowed_prefixes_pattern.search(line):  # Skip lines not matching the allowed patterns
            continue
        # # Delete incomplete sentences
        # if not re.search(r'\)$', line):
        #     continue
        # Add missing full stops
        if not re.search(r'\.$', line) and re.search(r'\)$', line):
            line += '.'
        # Remove dots between words
        line = re.sub(r'(?<=\w)\.(?=\w)', '', line)
        # Convert all numeric digits to their corresponding words
        line = convert_number_to_words(line)
        # remove special characters
        line = remove_unnecessary_characters(line)
        # Lowercase conversion while keeping ASP keywords or variables intact
        line = re.sub(r'(?<![A-Z])([A-Z]+)(?![A-Z])', lambda x: x.group(1).lower(), line)
        # Ensure only the final period is kept
        line = remove_all_but_final_period(line)  
        filtered_lines.append(line)
    return '\n'.join(filtered_lines)
        

def apply_filter_to_asp_code(asp_code_file, output_file):
    with open(asp_code_file, 'r') as file:
        asp_code = file.read()

    filtered_code = filter_asp_code(asp_code)

    with open(output_file, 'w') as file:
        file.write(filtered_code)
    
    print(f"Filtered ASP code has been written to {output_file}")

In [27]:
def prompt_model(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",) :
    
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    
    if tuned_model_name:
        model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    
    return(response.text)

In [28]:
def generating_asp_from_argument(argument):
    prompt = add_argument("Prompt to generate ASP code.txt", argument)

    response_text = prompt_model(
        project_id="aerial-grid-415517",
        model_name="text-bison-32k",
        temperature=0.2,
        max_decode_steps=8100,
        top_p=0.8,
        top_k=40,
        content=prompt,
        location="us-central1"
    )

    # Assuming the filter function modifies "filter.txt" directly,
    # first write the response to this file.
    with open("unfiltered asp code.txt", "w") as filt:
        filt.write(response_text)
    
    # Append the filtered content to "PaLM output.txt"
    # This avoids directly writing unfiltered response text and then filtered content,
    # streamlining the process to only include the desired output.
    with open("unfiltered asp code.txt", "r") as filt, open("ASP output.txt", 'a') as p:
        filtered_content = filt.read()
        p.write(filtered_content + "\n")
        print("ASP code generated")
    
    # Apply the filter to clean up "filter.txt"
apply_filter_to_asp_code("ASP output.txt","filtered asp code.txt")

Filtered ASP code has been written to filtered asp code.txt


In [29]:
def run_with_multiple_arguments(arguments_file):
    with open(arguments_file, 'r') as arguments:
        for argument in arguments:
            generating_asp_from_argument(argument)

In [30]:
def empty_file(file):
    with open(file, 'w'):
        pass

In [31]:
def clingo_compiling_check(file):
    # Clingo program
    program = clingo.Control()
    # Reading ASP code
    with open(file, 'r') as f:
        asp_code = f.read()
    # Load ASP code
    program.add('base', [], asp_code)
    # Compiling and solving
    try:
        program.ground([("base", [])])
        program.solve()
        return True
    except Exception as e:
        # Returns False if the ASP code has a wrong syntax and is not compiling
        return False

In [32]:
def solve_asp(file_path):
    # Read the ASP code from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        asp_code = file.read()

    # Create a new clingo Control object
    solver = clingo.Control()
    
    # Add the ASP code to the solver
    solver.add("base", [], asp_code)
    
    # Ground the program
    solver.ground([("base", [])])

    # Open the file where the solutions will be saved
    empty_file("clingo_output.txt")
    with open("clingo_output.txt", 'w') as solution_file:
        
        # Define the on_model function to write models to the file
        def on_model(model):
            solution_file.write(f"Solution: {model}\n")
        
        # Solve the ASP program, calling on_model for each solution
        result = solver.solve(on_model=on_model)
        
        # Check if a solution exists and write the outcome to the file
        if result.satisfiable:
            print("A solution exists. Check 'clingo_output.txt' for details.")
        else:
            solution_file.write("No solution exists.\n")
            print("No solution exists. Check 'clingo_output.txt' for details.")

In [33]:
# def generate_suggestions(asp_file):
#      input_before = ""
#      while_loop = True
#      while while_loop:
#          user_input = input("User: " + input_before)
#          if user_input.upper() == "STOP":
#              break
        
#         # Reading the prompt template for generating keywords
#          with open('Prompt for Keywords.txt', 'r') as file:
#              prompt = file.read().replace("$$$$$$$$$$$$$$$$$$$$", user_input)
        
# #         # Generate keywords using the prompt_model function
#          response_text = prompt_model(
#              project_id="aerial-grid-415517",
#              model_name="text-bison-32k",
#              temperature=0.2,
#              max_decode_steps=8100,
#              top_p=0.8,
#              top_k=40,
#              content=prompt,
#              location="us-central1"
#          )
        
#          # Extracting keywords from the generated text
#          keywords = response_text.split(', ')
#          print(keywords)
        
#          empty_file("clingo_output.txt")
#          for keyword in keywords:
#              print(keyword)
#              with open(asp_file, "r") as aspf:
#                  aspf_content = aspf.read()
#                  # Assuming 'encoding' contains your ASP encoding with placeholders for keywords
#                  if "(" + keyword in aspf_content or " " + keyword in aspf_content:
#                      asp_code = aspf_content  # + encoding.replace("§§§", keyword)
#                      # Assuming get_clingo_output processes and writes to "clingo_output.txt"
#                      solve_asp(asp_code)
#                      print("KEYWORD FINISHED")

#          # Reading the updated "clingo_output.txt" for the next prompt
#          with open("clingo_output.txt", "r") as cof:
#              clingo_output = cof.read()

# #         # Preparing the prompt with clingo output and user input for the next suggestion
#          with open("autocomplete_with_clingo.txt", "r") as acwt:
#              prompt = acwt.read()
#              prompt_with_clingo = prompt.replace("????????????????????", clingo_output)
#              prompt_with_clingo_and_text = prompt_with_clingo.replace("!!!!!!!!!!!!!!!!!!!!", user_input)
        
# #         # Generate suggestions using the updated prompt
#          response_text = prompt_model(
#              project_id="aerial-grid-415517",
#              model_name="text-bison-32k",
#              temperature=0.2,
#              max_decode_steps=8100,
#              top_p=0.8,
#              top_k=40,
#              content=prompt_with_clingo_and_text,
#              location="us-central1"
#          )
        
#          print(response_text)
#          input_before += (user_input + " ")

In [36]:
def generate_suggestions(asp_file):
    input_before = ""
    while_loop = True
    while while_loop:
        user_input = input("Please provide your input: ")
        if user_input.upper() == "STOP":
            break
        
        # Reading the prompt template for generating keywords
        with open('Prompt for Keywords.txt', 'r') as file:
            prompt = file.read().replace("$$$$$$$$$$$$$$$$$$$$", user_input)
        
        # Generate keywords using the prompt_model function
        response_text = prompt_model(
            project_id="aerial-grid-415517",
            model_name="text-bison-32k",
            temperature=0.2,
            max_decode_steps=8100,
            top_p=0.8,
            top_k=40,
            content=prompt,
            location="us-central1"
        )
        
        # Extracting keywords from the generated text
        print("Keywords Identified")
        keywords = response_text.split(', ')
        # print(keywords)

        empty_file("clingo_output.txt")
        for keyword in keywords:
            print(keyword)
            with open(asp_file, "r") as aspf:
                aspf_content = aspf.read()
                # Assuming 'encoding' contains your ASP encoding with placeholders for keywords
                if "(" + keyword in aspf_content or " " + keyword in aspf_content:
                    asp_code = aspf_content  # + encoding.replace("§§§", keyword)
                    # Assuming get_clingo_output processes and writes to "clingo_output.txt"
                    solve_asp(asp_code)
                    # print("KEYWORD FINISHED")

        # Reading the updated "clingo_output.txt" for the next prompt
        with open("clingo_output.txt", "r") as cof:
            clingo_output = cof.read()

        # Preparing the prompt with clingo output and user input for the next suggestion
        with open("autocomplete_with_clingo.txt", "r") as acwt:
            prompt = acwt.read()
            prompt_with_clingo = prompt.replace("????????????????????", clingo_output)
            prompt_with_clingo_and_text = prompt_with_clingo.replace("!!!!!!!!!!!!!!!!!!!!", user_input)

            # Generate suggestions using the updated prompt
        response_text = prompt_model(
            project_id="aerial-grid-415517",
            model_name="text-bison-32k",
            temperature=0.2,
            max_decode_steps=8100,
            top_p=0.8,
            top_k=40,
            content=prompt_with_clingo_and_text,
            location="us-central1"
        )
        
        if "Positive:" in response_text:
            print("\nPositive Aspects")
            start_pos = response_text.find("Positive:") + len("Positive:")
            end_pos = response_text.find("Negative:") if "Negative:" in response_text else len(response_text)
            positive_aspects = response_text[start_pos:end_pos].strip()
    
    # Assuming aspects are listed in lines
        for aspect in positive_aspects.split('\n'):
            if aspect.strip():  # Ensure the line is not empty
                print(f" {aspect.strip()}")

        if "Negative:" in response_text:
                print("\nNegative Aspects")
                start_pos = response_text.find("Negative:") + len("Negative:")
                negative_aspects = response_text[start_pos:].strip()
    # Assuming aspects are listed in lines
        for aspect in negative_aspects.split('\n'):
            if aspect.strip():  # Ensure the line is not empty
                print(f" {aspect.strip()}")

    input_before += (user_input + " ")

In [34]:
# empty_file("ASP output.txt")
# run_with_multiple_arguments("4arguments.txt")

ASP code generated
ASP code generated
ASP code generated
ASP code generated


In [38]:
# generate_suggestions("filtered asp code.txt")
# # type "stop" to discontinue

In [39]:
def main():
    empty_file("ASP output.txt")
    run_with_multiple_arguments("4arguments.txt")
    generate_suggestions("filtered asp code.txt")
    # type "stop" to discontinue